# Keras model evaluation workflow

## Imports

In [1]:
import sys
import os
sys.path.append(os.path.split(os.path.split(os.getcwd())[0])[0])
config_filepath ="./evaluate_config_generator.json"

In [2]:
import uuid
import json
import datetime
import getpass
import os

from mercury_ml.common import utils
from mercury_ml.common import tasks
from mercury_ml.common import containers as common_containers
from mercury_ml.keras import containers as keras_containers

Using TensorFlow backend.


## Helpers

These functions will help with the flow of this particular notebook

In [3]:
def print_data_bunch(data_bunch):

    for data_set_name, data_set in data_bunch.__dict__.items():
        print("{} <{}>".format(data_set_name, type(data_set).__name__))
        for data_wrapper_name, data_wrapper in data_set.__dict__.items():
            print("  {} <{}>".format(data_wrapper_name, type(data_wrapper).__name__))
        print()
        
def maybe_transform(data_bunch, pre_execution_parameters):
    if pre_execution_parameters:
        return data_bunch.transform(**pre_execution_parameters)
    else:
        return data_bunch
        
def print_dict(d):
    print(json.dumps(d, indent=2))

def get_installed_packages():
    import pip
    try:
        from pip._internal.operations import freeze
    except ImportError:  # pip < 10.0
        from pip.operations import freeze

    packages = []
    for p in freeze.freeze():
        packages.append(p)

    return packages

## Config

#### Load config

In [4]:
config = utils.load_referenced_json_config(config_filepath)

In [5]:
print_dict(config)

{
  "global_references": {
    "number_of_classes": 2,
    "batch_size": 2,
    "labels": [
      "cat",
      "dog"
    ]
  },
  "meta_info": {
    "ml_engine": "keras (tensorflow)",
    "model_purpose": "test_generator",
    "model_id": "caa1f315",
    "model_object_name": "{model_purpose}__{model_id}",
    "data_bunch_name": "images_456",
    "evaluation_session_id": "{evaluation_session_id}"
  },
  "init": {
    "read_source_data": {
      "name": "read_disk_keras_single_input_iterator"
    },
    "get_loss_function": {
      "name": "get_custom_loss"
    },
    "load_model": {
      "name": "load_hdf5"
    },
    "copy_from_local_to_remote": {
      "name": "copy_from_disk_to_disk",
      "params": {}
    },
    "copy_from_remote_to_local": {
      "name": "copy_from_disk_to_disk",
      "params": {}
    },
    "evaluate": {
      "name": "evaluate_generator"
    },
    "predict": {
      "name": "predict_generator"
    },
    "custom_metrics": {
      "names": [
        "evaluate

#### Set model_id

In [6]:
evaluation_session_id = str(uuid.uuid4().hex)[:8]

#### Update config

The function `utils.recursively_update_config(config, string_formatting_dict)` allows us to use string formatting to replace placeholder strings with acctual values.

for example: 

```python
>>> config = {"some_value": "some_string_{some_placeholder}"}
>>> string_formatting_dict = {"some_placeholder": "ABC"}
>>> utils.recursively_update_config(config, string_formatting_dict)
>>> print(config)
{"some_value": "some_string_ABC}"}
```



First update `config["meta_info"]`

In [7]:
utils.recursively_update_config(config["meta_info"], {
    "evaluation_session_id": evaluation_session_id,
    "model_id": config["meta_info"]["model_id"],
    "model_purpose": config["meta_info"]["model_purpose"]
})

Then use `config["meta_info"]` to update the rest.

In [8]:
utils.recursively_update_config(config, config["meta_info"])

## Session

Create a small dictionary with the session information. This will later be stored as a dictionary artifact with all the key run infomration

In [9]:
evaluation_session = {
    "time_stamp": datetime.datetime.utcnow().isoformat()[:-3] + "Z",
    "run_by": getpass.getuser(),
    "meta_info": config["meta_info"],
    "installed_packages": get_installed_packages()
}

In [10]:
print("Session info")
print(json.dumps(evaluation_session, indent=2))

Session info
{
  "time_stamp": "2019-02-01T13:44:11.081Z",
  "run_by": "karl.schriek",
  "meta_info": {
    "ml_engine": "keras (tensorflow)",
    "model_purpose": "test_generator",
    "model_id": "caa1f315",
    "model_object_name": "test_generator__caa1f315",
    "data_bunch_name": "images_456",
    "evaluation_session_id": "8e637351"
  },
  "installed_packages": [
    "absl-py==0.2.2",
    "alabaster==0.7.10",
    "apipkg==1.5",
    "astor==0.6.2",
    "astroid==1.6.5",
    "atomicwrites==1.1.5",
    "attrs==18.1.0",
    "autopep8==1.3.5",
    "Babel==2.6.0",
    "backcall==0.1.0",
    "better-apidoc==0.1.4",
    "bleach==3.1.0",
    "boto==2.49.0",
    "boto3==1.7.37",
    "botocore==1.10.37",
    "cachetools==2.1.0",
    "certifi==2018.11.29",
    "cffi==1.11.5",
    "chardet==3.0.4",
    "colorama==0.3.9",
    "cx-Oracle==6.3.1",
    "decorator==4.3.0",
    "dependency-injector==3.12.0",
    "docutils==0.14",
    "entrypoints==0.2.3",
    "execnet==1.5.0",
    "future==0.16.0",


## Initialization

These are the functions or classes we will be using in this workflow. We get / instatiate them all at the beginning using parameters under `config["initialization"]`.

Here we use mainly use `getattr` to fetch them via the `containers` module based on a string input in the config file. Providers could however also be fetched directly. The following three methods are all equivalent:

```python
# 1. (what we are using in this notebook)
from ml_workflow.common import containers as common_containers
source_reader=getattr(common_containers.SourceReaders, "read_pandas_data_set")

# 2. 
from ml_workflow.common import containers as common_containers
source_reader=common_containers.SourceReaders.read_pandas_data_set

# 3.
from ml_workflow.common.providers.source_reading import read_pandas_data_set
source_reader=read_pandas_data_set
```


### Helpers

These helper functions will create instantiate class providers (`create_and_log`) or fetch function providers (`get_and_log`) based on the parameters provided

In [11]:
def create_and_log(container, class_name, params):
    provider = getattr(container, class_name)(**params)
    print("{}.{}".format(container.__name__, class_name))
    print("params: ", json.dumps(params, indent=2))
    return provider

def get_and_log(container, function_name):
    provider = getattr(container, function_name)
    print("{}.{}".format(container.__name__, function_name))
    return provider

### Common

These are providers that are universally relevant, regardless of which Machine Learning engine is used.

In [12]:
# a function for storing dictionary artifacts to local disk
store_artifact_locally = get_and_log(common_containers.LocalArtifactStorers,
                                     config["init"]["store_artifact_locally"]["name"])

LocalArtifactStorers.store_dict_json


In [13]:
# a function for storing data-frame-like artifacts to local disk
store_prediction_artifact_locally = get_and_log(common_containers.LocalArtifactStorers,
                                                config["init"]["store_prediction_artifact_locally"]["name"])

LocalArtifactStorers.store_pandas_pickle


In [14]:
# a function for copy artifacts from local disk to a remote store
copy_from_local_to_remote = get_and_log(common_containers.ArtifactCopiers, config["init"]["copy_from_local_to_remote"]["name"])

ArtifactCopiers.copy_from_disk_to_disk


In [15]:
# a function for copy artifacts from remote store to a local disk
copy_from_remote_to_local = get_and_log(common_containers.ArtifactCopiers, config["init"]["copy_from_remote_to_local"]["name"])

ArtifactCopiers.copy_from_disk_to_disk


In [16]:
# a function for reading source data. When called it will return an instance of type DataBunch 
read_source_data_set = get_and_log(common_containers.SourceReaders, config["init"]["read_source_data"]["name"])

SourceReaders.read_disk_keras_single_input_iterator


In [17]:
# a dictionary of functions that calculate custom metrics
custom_metrics_dict = {
    custom_metric_name: get_and_log(common_containers.CustomMetrics, custom_metric_name) for custom_metric_name in config["init"]["custom_metrics"]["names"]
}


CustomMetrics.evaluate_numpy_auc
CustomMetrics.evaluate_numpy_micro_auc


In [18]:
# a dictionary of functions that calculate custom label metrics
custom_label_metrics_dict = {
    custom_label_metric_name: get_and_log(common_containers.CustomLabelMetrics, custom_label_metric_name) for custom_label_metric_name in config["init"]["custom_label_metrics"]["names"]
}


CustomLabelMetrics.evaluate_numpy_accuracy
CustomLabelMetrics.evaluate_numpy_confusion_matrix


### Keras

In [19]:
# a function that returns a keras loss function
get_loss_function = get_and_log(keras_containers.LossFunctionFetchers, 
                                config["init"]["get_loss_function"]["name"])

LossFunctionFetchers.get_custom_loss


In [20]:
# a function that loads a Keras model
load_model = get_and_log(keras_containers.ModelLoaders, config["init"]["load_model"]["name"])

ModelLoaders.load_hdf5


In [21]:
# a function for evaluating keras metrics
evaluate = get_and_log(keras_containers.ModelEvaluators, config["init"]["evaluate"]["name"])

ModelEvaluators.evaluate_generator


In [22]:
# a function that predictions using a keras model
predict = get_and_log(keras_containers.PredictionFunctions, config["init"]["predict"]["name"])

PredictionFunctions.predict_generator


## Execution

Here we use the providers defined above to execute various tasks

### Save (formatted) config

In [23]:
tasks.store_artifacts(store_artifact_locally, copy_from_local_to_remote, config,
                      **config["exec"]["save_formatted_config"]["params"])

In [24]:
print("Config stored with following parameters")
print_dict(config["exec"]["save_formatted_config"]["params"])

Config stored with following parameters
{
  "local_dir": "./example_results/local/model_purpose/test_generator/data_bunch/images_456/model_id/caa1f315/evaluation_session/8e637351",
  "remote_dir": "./example_results/remote/model_purpose/test_generator/data_bunch/images_456/model_id/caa1f315/evaluation_session/8e637351",
  "filename": "config_formatted.json"
}


### Save Session

##### Save session info

In [25]:
tasks.store_artifacts(store_artifact_locally, copy_from_local_to_remote, evaluation_session,
                      **config["exec"]["save_evaluation_session"]["params"])

In [26]:
print("Session dictionary stored with following parameters")
print_dict(config["exec"]["save_evaluation_session"]["params"])

Session dictionary stored with following parameters
{
  "local_dir": "./example_results/local/model_purpose/test_generator/data_bunch/images_456/model_id/caa1f315/evaluation_session/8e637351",
  "remote_dir": "./example_results/remote/model_purpose/test_generator/data_bunch/images_456/model_id/caa1f315/evaluation_session/8e637351",
  "filename": "session.json"
}


##### Save session artifacts

In [27]:
for filename in config["exec"]["save_evaluation_session_artifacts"]["params"]["filenames"]:
    # save to local artifact store
    common_containers.ArtifactCopiers.copy_from_disk_to_disk(source_dir=os.getcwd(),
                                                   target_dir=config["exec"]["save_evaluation_session_artifacts"]["params"]["local_dir"],
                                                   filename=filename,
                                                   overwrite=False,
                                                   delete_source=False)

    # copy to remote artifact store
    copy_from_local_to_remote(source_dir=config["exec"]["save_evaluation_session_artifacts"]["params"]["local_dir"],
                              target_dir=config["exec"]["save_evaluation_session_artifacts"]["params"]["remote_dir"],
                              filename=filename,
                              overwrite=False,
                              delete_source=False)
    

In [28]:
print("Session artifacts stored with following parameters")
print_dict(config["exec"]["save_evaluation_session_artifacts"]["params"])

Session artifacts stored with following parameters
{
  "local_dir": "./example_results/local/model_purpose/test_generator/data_bunch/images_456/model_id/caa1f315/evaluation_session/8e637351",
  "remote_dir": "./example_results/remote/model_purpose/test_generator/data_bunch/images_456/model_id/caa1f315/evaluation_session/8e637351",
  "filenames": [
    "evaluate.ipynb",
    "evaluate_config_generator.json"
  ]
}


### Get source data

In [29]:
data_bunch_source = tasks.read_test_data_bunch(read_source_data_set,**config["exec"]["read_source_data"]["params"] )
print("Source data read using following parameters: \n")
print_dict(config["exec"]["read_source_data"]["params"])

Found 6 images belonging to 2 classes.
Source data read using following parameters: 

{
  "test_params": {
    "generator_params": {},
    "iterator_params": {
      "directory": "./example_data/images_456/test",
      "batch_size": 2,
      "class_mode": "categorical",
      "color_mode": "rgb",
      "seed": 12345,
      "shuffle": false,
      "target_size": [
        10,
        10
      ]
    }
  }
}


In [30]:
print("Read data_bunch consists of: \n")
print_data_bunch(data_bunch_source)

Read data_bunch consists of: 

test <DataSet>
  features <KerasIteratorFeaturesDataWrapper>
  targets <KerasIteratorTargetsDataWrapper>
  index <KerasIteratorIndexDataWrapper>



### Load Model

##### Get custom loss function

In [31]:
if config["init"]["get_loss_function"]["name"] == "get_custom_loss":
    loss = get_loss_function(**config["exec"]["get_loss_function"]["params"])
    custom_objects = {loss.__name__: loss}
else:
    custom_objects = None

In [32]:
print(custom_objects)

{'mock_loss': <function get_mock_loss_function.<locals>.mock_loss at 0x0000021198EE9400>}


In [33]:
model = tasks.load_model(load_model=load_model,
                         copy_from_remote_to_local=copy_from_remote_to_local,
                         custom_objects = custom_objects,
                         **config["exec"]["load_model"]
                        )

### Evaluate metrics

##### Transform data

In [34]:
data_bunch_metrics = maybe_transform(data_bunch_source, config["exec"]["evaluate"].get("pre_execution_transformation"))

print("Data transformed with following parameters: \n")
print_dict(config["exec"]["evaluate"].get("pre_execution_transformation"))

Data transformed with following parameters: 

null


In [35]:
print("Transformed data_bunch consists of: \n")
print_data_bunch(data_bunch_metrics)

Transformed data_bunch consists of: 

test <DataSet>
  features <KerasIteratorFeaturesDataWrapper>
  targets <KerasIteratorTargetsDataWrapper>
  index <KerasIteratorIndexDataWrapper>



##### Calculate metrics

In [36]:
metrics = {}
for data_set_name in config["exec"]["evaluate"]["data_set_names"]:
    data_set = getattr(data_bunch_metrics, data_set_name)
    metrics[data_set_name] = evaluate(model, data_set, **config["exec"]["evaluate"]["params"])

In [37]:
print("Resulting metrics: \n")
print_dict(metrics)

Resulting metrics: 

{
  "test": {
    "loss": 8.059051195780436,
    "acc": 0.5
  }
}


### Save metrics

In [38]:
#TODO existing metrics should be updated, not overwritten!

for data_set_name, params in config["exec"]["save_metrics"]["data_sets"].items():
    tasks.store_artifacts(store_artifact_locally, copy_from_local_to_remote, metrics[data_set_name],
                          **params)

### Predict

##### Transform data

In [39]:
data_bunch_predict = maybe_transform(data_bunch_metrics, config["exec"]["predict"].get("pre_execution_transformation"))

print("Data transformed with following parameters: \n")
print_dict(config["exec"]["predict"].get("pre_execution_transformation"))

Data transformed with following parameters: 

null


In [40]:
print("Transformed data_bunch consists of: \n")
print_data_bunch(data_bunch_predict)

Transformed data_bunch consists of: 

test <DataSet>
  features <KerasIteratorFeaturesDataWrapper>
  targets <KerasIteratorTargetsDataWrapper>
  index <KerasIteratorIndexDataWrapper>



##### Perform prediction

In [41]:
for data_set_name in config["exec"]["predict"]["data_set_names"]:
    data_set = getattr(data_bunch_predict, data_set_name)
    data_set.predictions = predict(model=model, data_set=data_set, **config["exec"]["predict"]["params"])

In [42]:
print("Data predicted with following parameters: \n")
print_dict(config["exec"]["predict"].get("params"))


Data predicted with following parameters: 

{}


### Evaluate custom metrics

##### Transform data

In [43]:
data_bunch_custom_metrics = maybe_transform(data_bunch_predict, 
                                            config["exec"]["evaluate_custom_metrics"].get("pre_execution_transformation"))

In [44]:
print("Data transformed with following parameters: \n")
print_dict(config["exec"]["evaluate_custom_metrics"].get("pre_execution_transformation"))

Data transformed with following parameters: 

{
  "data_set_names": [
    "test"
  ],
  "params": {
    "transform_to": "numpy",
    "data_wrapper_params": {
      "predictions": {},
      "index": {},
      "targets": {}
    }
  }
}


In [45]:
print("Transformed data_bunch consists of: \n")
print_data_bunch(data_bunch_custom_metrics)


Transformed data_bunch consists of: 

test <DataSet>
  predictions <NumpyDataWrapper>
  index <NumpyDataWrapper>
  targets <NumpyDataWrapper>



##### Calculate custom metrics


In [46]:
custom_metrics = {}
for data_set_name in config["exec"]["evaluate_custom_metrics"]["data_set_names"]:
    data_set = getattr(data_bunch_custom_metrics, data_set_name)
    custom_metrics[data_set_name]  = tasks.evaluate_metrics(data_set, custom_metrics_dict)


In [47]:
print("Resulting custom metrics: \n")
print_dict(custom_metrics)


Resulting custom metrics: 

{
  "test": {
    "evaluate_numpy_auc": 0.4722222222222222,
    "evaluate_numpy_micro_auc": 0.45833333333333337
  }
}


##### Calculate custom label metrics

In [48]:
custom_label_metrics = {}
for data_set_name in config["exec"]["evaluate_custom_label_metrics"]["data_set_names"]:
    data_set = getattr(data_bunch_custom_metrics, data_set_name)
    custom_label_metrics[data_set_name] = tasks.evaluate_label_metrics(data_set, custom_label_metrics_dict)

In [49]:
print("Resulting custom label metrics: \n")
print_dict(custom_label_metrics)

Resulting custom label metrics: 

{
  "test": {
    "Accuracy": {
      "cat": 0.5,
      "dog": 0.5
    },
    "ConfMat_Count_cat": {
      "cat": 2,
      "dog": 2
    },
    "ConfMat_Rate_cat": {
      "cat": 0.6666666666666666,
      "dog": 0.6666666666666666
    },
    "ConfMat_Count_dog": {
      "cat": 1,
      "dog": 1
    },
    "ConfMat_Rate_dog": {
      "cat": 0.3333333333333333,
      "dog": 0.3333333333333333
    }
  }
}


In [50]:
for data_set_name, params in config["exec"]["save_custom_metrics"]["data_sets"].items():
    tasks.store_artifacts(store_artifact_locally, copy_from_local_to_remote,
                          custom_metrics[data_set_name], **params)

In [51]:
print("Custom metrics saved with following parameters: \n")
print_dict(config["exec"]["save_custom_metrics"])

Custom metrics saved with following parameters: 

{
  "data_sets": {
    "test": {
      "local_dir": "./example_results/local/model_purpose/test_generator/data_bunch/images_456/model_id/caa1f315/metrics/test",
      "remote_dir": "./example_results/remote/model_purpose/test_generator/data_bunch/images_456/model_id/caa1f315/metrics/test",
      "filename": "test_generator__caa1f315__test__custom_metrics"
    }
  }
}


In [52]:
for data_set_name, params in config["exec"]["save_custom_label_metrics"]["data_sets"].items():
    tasks.store_artifacts(store_artifact_locally, copy_from_local_to_remote,
                          custom_label_metrics[data_set_name], **params)

In [53]:
print("Custom label metrics saved with following parameters: \n")
print_dict(config["exec"]["save_custom_label_metrics"])

Custom label metrics saved with following parameters: 

{
  "data_sets": {
    "test": {
      "local_dir": "./example_results/local/model_purpose/test_generator/data_bunch/images_456/model_id/caa1f315/metrics/test",
      "remote_dir": "./example_results/remote/model_purpose/test_generator/data_bunch/images_456/model_id/caa1f315/metrics/test",
      "filename": "test_generator__caa1f315__test__custom_label_metrics"
    }
  }
}


### Prepare predictions for storage

##### Transform data

In [54]:
data_bunch_prediction_preparation = maybe_transform(data_bunch_predict, 
                                                    config["exec"]["prepare_predictions_for_storage"].get("pre_execution_transformation"))

In [55]:
print("Transformed data_bunch consists of: \n")
print_data_bunch(data_bunch_prediction_preparation)

Transformed data_bunch consists of: 

test <DataSet>
  predictions <PandasDataWrapper>
  index <PandasDataWrapper>
  targets <PandasDataWrapper>



##### Prepare predictions and targets

In [56]:
for data_set_name in config["exec"]["prepare_predictions_for_storage"]["data_set_names"]:
    data_set = getattr(data_bunch_prediction_preparation, data_set_name)
    data_set.add_data_wrapper_via_concatenate(**config["exec"]["prepare_predictions_for_storage"]["params"]["predictions"])
    data_set.add_data_wrapper_via_concatenate(**config["exec"]["prepare_predictions_for_storage"]["params"]["targets"])

### Save predictions

##### Transform data

In [57]:
data_bunch_prediction_storage = maybe_transform(data_bunch_prediction_preparation, 
                                                config["exec"]["save_predictions"].get("pre_execution_transformation"))

In [58]:
print("Transformed data_bunch consists of: \n")
print_data_bunch(data_bunch_prediction_storage)

Transformed data_bunch consists of: 

test <DataSet>
  predictions <PandasDataWrapper>
  index <PandasDataWrapper>
  targets <PandasDataWrapper>
  predictions_for_storage <PandasDataWrapper>
  targets_for_storage <PandasDataWrapper>



##### Save predictions

In [59]:
for data_set_name, data_set_params in config["exec"]["save_predictions"]["data_sets"].items():
    data_set = getattr(data_bunch_prediction_storage, data_set_name)
    data_wrapper = getattr(data_set, data_set_params["data_wrapper_name"])
    
    data_to_store = data_wrapper.underlying
   
    tasks.store_artifacts(store_prediction_artifact_locally, copy_from_local_to_remote,
                          data_to_store, **data_set_params["params"])

In [60]:
print("Predictions saved with following parameters: \n")
print_dict(config["exec"]["save_predictions"])

Predictions saved with following parameters: 

{
  "data_sets": {
    "test": {
      "data_wrapper_name": "predictions_for_storage",
      "params": {
        "local_dir": "./example_results/local/model_purpose/test_generator/data_bunch/images_456/model_id/caa1f315/predictions/test",
        "remote_dir": "./example_results/remote/model_purpose/test_generator/data_bunch/images_456/model_id/caa1f315/predictions/test",
        "filename": "test_generator__caa1f315__test__predictions"
      }
    }
  }
}


##### Save targets

In [61]:
for data_set_name, data_set_params in config["exec"]["save_targets"]["data_sets"].items():
    data_set = getattr(data_bunch_prediction_storage, data_set_name)
    data_wrapper = getattr(data_set, data_set_params["data_wrapper_name"])
    
    data_to_store = data_wrapper.underlying
   
    tasks.store_artifacts(store_prediction_artifact_locally, copy_from_local_to_remote,
                          data_to_store, **data_set_params["params"])

In [62]:
print("Targets saved with following parameters: \n")
print_dict(config["exec"]["save_targets"])

Targets saved with following parameters: 

{
  "data_sets": {
    "test": {
      "data_wrapper_name": "targets_for_storage",
      "params": {
        "local_dir": "./example_results/local/model_purpose/test_generator/data_bunch/images_456/model_id/caa1f315/predictions/test",
        "remote_dir": "./example_results/remote/model_purpose/test_generator/data_bunch/images_456/model_id/caa1f315/predictions/test",
        "filename": "test_generator__caa1f315__test__targets"
      }
    }
  }
}
